In [1]:
import numpy as np
import pandas as pd
from dask import dataframe as dd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

stemmer = SnowballStemmer("english")

pd.set_option('display.max_columns', 100)

# Government Spend Analysis

## Step 1: Examine column headers to identify columns of interest

In [2]:
test = pd.read_csv('FY2016_All_Contracts_Full_20210608_1.csv', nrows = 5)

test

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,...,manufacturer_of_goods,veterinary_hospital,hispanic_servicing_institution,receives_contracts,receives_financial_assistance,receives_contracts_and_financial_assistance,airport_authority,council_of_governments,housing_authorities_public_tribal,interstate_entity,planning_commission,port_authority,transit_authority,subchapter_scorporation,limited_liability_corporation,foreign_owned,for_profit_organization,nonprofit_organization,other_not_for_profit_organization,the_ability_one_program,private_university_or_college,state_controlled_institution_of_higher_learning,1862_land_grant_college,1890_land_grant_college,1994_land_grant_college,minority_institution,historically_black_college,tribal_college,alaskan_native_servicing_institution,native_hawaiian_servicing_institution,school_of_forestry,veterinary_college,dot_certified_disadvantage,self_certified_small_disadvantaged_business,small_disadvantaged_business,c8a_program_participant,historically_underutilized_business_zone_hubzone_firm,sba_certified_8a_joint_venture,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,1,0,0,8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",DU205NC16D02,0,799750.00,546621.23,799750.00,546621.23,799750.00,546621.23,NaN,NaN,NaN,2016-09-30,2016,2016-09-30 00:00:00,2021-09-29 00:00:00,2021-09-29 00:00:00,NaN,NaN,86,DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT (HUD),8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",86545A,DEPT HUD-CHIEF PROCUREMENT OFFICER,86,DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT (HUD),8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",8630AX,DEPT OF HOUSING URBAN DEVELOPMENT,086-2016/2017-0183-000,086-0183,25.3: Other goods and services from Federal so...,0709: ADMINISTRATIVE EXPENSES,X,NOT APPLICABLE,NaN,NaN,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,NaN,831854596,...,f,f,f,f,f,t,f,f,f,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,t,f,t,f,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_000...,2019-02-12 12:16:44
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,1,5,0,6800,ENVIRONMENTAL PROTECTION AGENCY,EPR91502,0,0.00,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,2016-09-30,2016,2016-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,NaN,NaN,68,ENVIRO

### Columns of interest:

* contract_transaction_unique_key
* contract_award_unique_key
* awarding_agency_code
* awarding_sub_agency_code
* awarding_office_code
* funding_agency_code
* funding_sub_agency_code
* funding_office_code
* recipient_duns
* recipient_name
* recipient_doing_business_as_name
* recipient_parent_duns


## Step 2: Load data

Using dask with only columns of interest included.

In [3]:
columns = ['contract_transaction_unique_key', 'contract_award_unique_key', 'awarding_agency_code', 'awarding_sub_agency_code',\
           'awarding_office_code', 'funding_agency_code', 'funding_sub_agency_code', 'funding_office_code', 'recipient_duns', \
           'recipient_name', 'recipient_doing_business_as_name', 'recipient_parent_duns']

data = dd.read_csv('*.csv', usecols=columns, dtype={'awarding_sub_agency_code': 'object', 'funding_sub_agency_code': 'object', \
                                                    'recipient_doing_business_as_name': 'object', 'recipient_parent_duns': 'object',\
                                                    'funding_agency_code': 'object', 'awarding_office_code': 'object', \
                                                    'funding_office_code': 'object', 'recipient_duns': 'object'})

data.head()

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,86,8600,86545A,086,8600,8630AX,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,831854596
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987
2,6800_6800_0001_20_EPR51502_0,CONT_AWD_0001_6800_EPR51502_6800,68,6800,68HE05,068,6800,68U000,079493355,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,NaN,079493355
3,6800_6800_0001_0_EPS21602_0,CONT_AWD_0001_6800_EPS21602_6800,68,6800,68HE02,068,6800,68R000,080265258,CAT-ENSOFTEK,NaN,080265258
4,6800_6800_0001_31_EPS51301_0,CONT_AWD_0001_6800_EPS51301_6800,68,6800,68HE05,068,6800,68U000,178599221,"TETRA TECH EM, INC.",NaN,045224250


### Step 2a: Save as parquet format

(So it's easier to reload if needed)

In [4]:
data.to_parquet('government_spend_dataset.parquet', engine='pyarrow')

(None,)

## Step 3: Get number of initial records

In [5]:
a = data.index.size.compute()

print("There are ", a, " initial records in this dataset.")

There are  4820290  initial records in this dataset.


## Step 4: Create list of unique suppliers (exact)

(and get number of unique suppliers)

In [6]:
suppliers_unique = data.recipient_name.unique().compute().reset_index()

print("There are ", len(suppliers_unique), " unique suppliers (exact match only).")

There are  153462  unique suppliers (exact match only).


In [7]:
suppliers_unique.head()

,index,recipient_name
0,0,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC"
1,1,INNOVATE INC
2,2,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...
3,3,CAT-ENSOFTEK
4,4,"TETRA TECH EM, INC."


### Step 4a: Does recipient_doing_business_as_name have any non-null values?

Answer: yes, hold off on this until next step

In [8]:
data.recipient_doing_business_as_name.dropna().compute()

23                MIT LINCOLN LABORATORY
29                                 U T C
39        SOFTWARE ENGINEERING INSTITUTE
52                    PACIFIC SCIENTIFIC
86                    PACIFIC SCIENTIFIC
                      ...               
18863    TOUCHSTONE NEURORECOVERY CENTER
18864                     PROVIDE A RIDE
18875             PETNET PHARMACEUTICALS
18882        MUSC DEPARTMENT OF MEDICINE
18895                 BIMBO BAKERIES USA
Name: recipient_doing_business_as_name, Length: 94562, dtype: object

## Step 5: Snowball stemmer

In [9]:
stop = set(stopwords.words('english'))

suppliers_unique['recipient_name_stemmed'] = suppliers_unique['recipient_name'].astype(str).str.split().\
apply(lambda x: [stemmer.stem(y) for y in x if stemmer.stem(y) not in stop and len(y) > 1])

suppliers_unique

,index,recipient_name,recipient_name_stemmed
0,0,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC","[mortgag, assist, acquisit, group,, llc]"
1,1,INNOVATE INC,"[innov, inc]"
2,2,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,"[ryba, marin, construct, co/terra, contract, s..."
3,3,CAT-ENSOFTEK,[cat-ensoftek]
4,4,"TETRA TECH EM, INC.","[tetra, tech, em,, inc.]"
...,...,...,...
153457,153457,"AZORMED - GESTAO AMBIENTAL ACOREANA, LDA","[azorm, gestao, ambient, acoreana,, lda]"
153458,153458,JEAN ROB & CO,"[jean, rob, co]"
153459,153459,"WOODS SERVICES, INC.","[wood, services,, inc.]"
153460,153460,FLOWERS & ASSOCIATES,"[flower, associ]"


In [11]:
suppliers_unique['lemma_str'] = suppliers_unique.recipient_name_stemmed.agg(lambda x: ','.join(map(str, x)))

suppliers_unique.lemma_str.value_counts().sort_values(ascending = False)

inc                                        68
corpor                                     29
enterpris                                  25
enterpris,inc                              22
incorpor                                   19
                                           ..
toshiba,medic,visual,system,europe,,ltd     1
link,medic,comput,inc                       1
weber,fire,safeti,equip,co.                 1
indian,nation,fiberopt,inc                  1
gari,bluemk                                 1
Name: lemma_str, Length: 152196, dtype: int64

## Step 6: Sentence embeddings

## Step 7: Cosine similarity

## Step 8: Evaluating accuracy

## Step 9: Newly answerable questions